# Gallery Example: M/MAP/1 Queue (Multiclass)

This example demonstrates a multiclass M/MAP/1 queueing system:
- **Arrivals**: Exponential (Poisson) for each class
- **Service**: MAP distribution per class
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS

Each class has its own Poisson arrival stream and MAP service distribution.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_mmap1_multiclass(map1=None, map2=None): Network {
    """Create multiclass M/MAP/1 queueing model"""
    if map1 is None:
        val map1 = MAP.rand(2)
        map1.set_mean(0.5)
    if map2 is None:
        val map2 = MAP.rand(3)
        map2.set_mean(0.5)
    
    val model = Network("M/MAP/1")
// Block 1: nodes
    val source = Source(model, "mySource")
    val queue = Queue(model, "myQueue", SchedStrategy.FCFS)
    val sink = Sink(model, "mySink")
// Block 2: classes
// Class 1: arrival rate adjusted for 35% of total load
    val oclass1 = OpenClass(model, "myClass1")
    source.setArrival(oclass1, Exp(0.35 / map1.get_mean()))
    queue.setService(oclass1, map1)
// Class 2: arrival rate adjusted for 15% of total load
    val oclass2 = OpenClass(model, "myClass2")
    source.setArrival(oclass2, Exp(0.15 / map2.get_mean()))
    queue.setService(oclass2, map2)
// Block 3: topology
    val P = model.initRoutingMatrix()
    P.addRoute(oclass1, source, queue, 1.0)
    P.addRoute(oclass1, queue, sink, 1.0)
    P.addRoute(oclass2, source, queue, 1.0)
    P.addRoute(oclass2, queue, sink, 1.0)
    model.link(P)
    
    return model
// Create the model
val model = gallery_mmap1_multiclass()

## About Multiclass M/MAP/1 Queues

This model features:
- Two job classes with different characteristics
- **Class 1**: 35% of total utilization with a 2-phase MAP service
- **Class 2**: 15% of total utilization with a 3-phase MAP service

Total utilization: ρ = 0.35 + 0.15 = 0.50

The arrival rates are computed as: λᵢ = ρᵢ / E[Sᵢ]
- Where ρᵢ is the class utilization target
- E[Sᵢ] is the mean service time for class i

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)